In [10]:
%pip install transformers torch spacy scikit-learn
!python -m spacy download en_core_web_sm


Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - ------------------------------------- 0.5/12.8 MB 621.2 kB/s eta 0:00:20
     - ------------------------------------- 0.5/12.8 MB 621.2 kB/s eta 0:00:20
     -- ------------------------------------ 0.8/12.8 MB 589.1 kB/s eta 0:00:21
     -- ------------------------------------ 0.8/12.8 MB 589.1 kB/s eta 0:00:21
     --- ----------------------------------- 1.0/12.8 MB 599.4 kB/s eta 0:00:20
     --- ----------------------------------- 1.0/12.8 MB 599.4 kB/s eta 0:00:20
     --- ----------------------------------- 1.3/12.8 MB 564.1 kB/s eta 0:00:21
     --- -----

In [11]:
import spacy
import torch
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from sklearn.preprocessing import LabelEncoder
import re

In [12]:
# Load SpaCy for NER and Lemmatization
nlp = spacy.load("en_core_web_sm")

In [13]:
# Load BERT-based sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [14]:
# Dummy intents and responses
intent_responses = {
    "check_balance": "Sure, I can help you check your balance. Please log in to your account.",
    "transaction_help": "Please provide the transaction ID or date for assistance.",
    "fraud_report": "We're sorry to hear that. Your case is now escalated to our fraud team.",
    "general_inquiry": "How can I help you today?",
    "unknown": "I'm not sure how to respond to that. Can you rephrase?"
}


In [15]:
# Intent recognition mockup model (replace with fine-tuned BERT in production)
class MockIntentClassifier:
    def __init__(self):
        self.labels = ["check_balance", "transaction_help", "fraud_report", "general_inquiry", "unknown"]
    
    def predict(self, text):
        text = text.lower()
        if "balance" in text:
            return "check_balance"
        elif "transaction" in text:
            return "transaction_help"
        elif "fraud" in text or "unauthorized" in text:
            return "fraud_report"
        elif "hello" in text or "help" in text:
            return "general_inquiry"
        else:
            return "unknown"

intent_model = MockIntentClassifier()

In [16]:

# Text preprocessing
def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

# Named Entity Recognition
def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

# Chatbot logic
def chatbot_response(user_input):
    cleaned_input = preprocess(user_input)
    
    # Sentiment analysis
    sentiment = sentiment_pipeline(user_input)[0]
    
    # Intent detection
    intent = intent_model.predict(cleaned_input)
    
    # Named entity recognition
    entities = extract_entities(user_input)

    response = intent_responses.get(intent, intent_responses["unknown"])
    
    return {
        "intent": intent,
        "response": response,
        "sentiment": sentiment,
        "entities": entities
    }

In [17]:
# Example interaction
if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            break
        reply = chatbot_response(user_input)
        print(f"\nBot Response: {reply['response']}")
        print(f"Intent: {reply['intent']} | Sentiment: {reply['sentiment']['label']} ({reply['sentiment']['score']:.2f})")
        print(f"Entities: {reply['entities']}\n")


Bot Response: Sure, I can help you check your balance. Please log in to your account.
Intent: check_balance | Sentiment: POSITIVE (1.00)
Entities: []


Bot Response: Sure, I can help you check your balance. Please log in to your account.
Intent: check_balance | Sentiment: POSITIVE (1.00)
Entities: []


Bot Response: I'm not sure how to respond to that. Can you rephrase?
Intent: unknown | Sentiment: POSITIVE (0.75)
Entities: []

